Ссылка на БД: https://tochno.st/datasets/bdmo


In [1]:
%%capture
!pip install osmnx pyrosm
!apt-get update -qq && apt-get install -y osmium-tool

In [2]:
import pandas as pd
import numpy as np
import json
from pyrosm import OSM, get_data
import requests
from shapely.geometry import LineString
from shapely.ops import polygonize, unary_union
import json
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Население (данные 2023 года)

Файл: data_Y48112015_year2023_112_v20241111



In [4]:
districts_area = {
    "Академический": 46.01,
    "Верх-Исетский": 219.79,
    "Железнодорожный": 125.65,
    "Кировский": 86.25,
    "Ленинский": 22.19,
    "Октябрьский": 158.6,
    "Орджоникидзевский": 99.3,
    "Чкаловский": 389.81
}

districts_pop = {
    "Академический": 125000,
    "Верх-Исетский": 240822,
    "Железнодорожный": 158675,
    "Кировский": 220749,
    "Ленинский": 222258,
    "Октябрьский": 151775,
    "Орджоникидзевский": 263820,
    "Чкаловский": 286277
}

In [16]:
# Добавим площадь
df = pd.DataFrame.from_dict(districts_area, orient='index', columns=['area']).reset_index()
df = df.rename({"index":"mun_district"}, axis=1)

df['population'] = df['mun_district'].apply(
    lambda x: districts_pop.get(x, None)
    )

df

,mun_district,area,population
0,Академический,46.01,125000
1,Верх-Исетский,219.79,240822
2,Железнодорожный,125.65,158675
3,Кировский,86.25,220749
4,Ленинский,22.19,222258
5,Октябрьский,158.60,151775
6,Орджоникидзевский,99.30,263820
7,Чкаловский,389.81,286277


### Плотность всего населения

По муниципальным округам (чел/км2)

In [17]:
# Плотность всего населения по округам
df["pop_dense"] = df["population"] / df['area']

In [18]:
# Начинаем составлять итоговый датасет
result = df
result

,mun_district,area,population,pop_dense
0,Академический,46.01,125000,2716.800696
1,Верх-Исетский,219.79,240822,1095.691342
2,Железнодорожный,125.65,158675,1262.833267
3,Кировский,86.25,220749,2559.408696
4,Ленинский,22.19,222258,10016.133393
5,Октябрьский,158.60,151775,956.967213
6,Орджоникидзевский,99.30,263820,2656.797583
7,Чкаловский,389.81,286277,734.401375


## Границы административных районов

Чтобы спроецировать данные на карте - необходимо получить координаты всех муниципальных районов Москвы

In [30]:
!wget -nc https://download.geofabrik.de/russia/ural-fed-district-latest.osm.pbf

--2025-06-13 13:08:26--  https://download.geofabrik.de/russia/ural-fed-district-latest.osm.pbf
Resolving download.geofabrik.de (download.geofabrik.de)... 95.217.63.98, 95.216.245.233, 95.217.45.61, ...
Connecting to download.geofabrik.de (download.geofabrik.de)|95.217.63.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 374671635 (357M) [application/octet-stream]
Saving to: ‘ural-fed-district-latest.osm.pbf’

ural-fed-district-l 100%[===================>] 357.31M  20.6MB/s    in 18s     

2025-06-13 13:08:44 (20.1 MB/s) - ‘ural-fed-district-latest.osm.pbf’ saved [374671635/374671635]



In [32]:
overpass_url = "https://overpass-api.de/api/interpreter"
query = """
[out:json][timeout:25];
area["ISO3166-2"="RU-SVE"]->.region;
(
  relation["place"="city"](area.region)["name"="Екатеринбург"];
  relation["place"="town"](area.region)["name"="Екатеринбург"];
);
out geom;
"""

resp = requests.get(overpass_url, params={"data": query})
data = resp.json()

In [34]:
# Создаем список всех полигонов районов
district_polygons = []
for element in data["elements"]:
    if element["type"] == "relation":
        members = element.get("members", [])
        lines = []
        for m in members:
            if m["role"] in ["outer", ""] and "geometry" in m:  # Некоторые роли могут быть пустыми
                coords = [(pt["lon"], pt["lat"]) for pt in m["geometry"]]
                if len(coords) > 1:  # Проверяем, что линия валидна
                    lines.append(LineString(coords))

        if lines:
            try:
                # Пробуем создать полигон из линий
                polygons = list(polygonize(lines))
                if polygons:
                    district_polygon = unary_union(polygons)
                    district_polygons.append({
                        "type": "Feature",
                        "properties": {"name": element["tags"].get("name", "")},
                        "geometry": json.loads(json.dumps(district_polygon.__geo_interface__))
                    })
            except:
                print(f"Ошибка при обработке района {element.get('id')}")

# Сохраняем все районы как FeatureCollection
geojson = {
    "type": "FeatureCollection",
    "features": district_polygons
}

with open("ekb_districts.geojson", "w", encoding="utf-8") as f:
    json.dump(geojson, f, ensure_ascii=False, indent=2)

print("Границы районов сохранены в ekb_districts.geojson")

Границы районов сохранены в ekb_districts.geojson


In [35]:
!osmium extract \
    --overwrite \
    --output-format pbf \
    --polygon ekb_districts.geojson \
    ural-fed-district-latest.osm.pbf \
    -o ekb-districts.osm.pbf

[======================================================================] 100% 


In [36]:
fp = "ekb-districts.osm.pbf"

osm = OSM(fp)

# Получаем все административные границы
boundaries = osm.get_boundaries()

In [37]:
print(boundaries.keys()), len(boundaries)

Index(['visible', 'admin_level', 'boundary', 'id', 'timestamp', 'version',
       'osm_type', 'geometry', 'addr:country', 'name', 'changeset', 'tags'],
      dtype='object')


(None, 23)

In [38]:
mun_geo = boundaries[["name", "geometry"]]
mun_geo.to_csv("mun_geo.csv")
# mun_geo = pd.read_csv("mun_geo.csv")

In [40]:
mun_geo = mun_geo.rename(columns={"name": "mun_district"})
mun_geo['mun_district'] = mun_geo['mun_district'].str.replace("ё", 'е')

patterns = ['городской округ ', 'район ', ' район']
for pattern in patterns:
        mun_geo['mun_district'] = mun_geo['mun_district'].str.replace(pattern, '')

mun_geo.head()

,mun_district,geometry
0,NaN,"LINESTRING (60.58189 56.75238, 60.58221 56.75253)"
1,NaN,"MULTILINESTRING ((60.58221 56.75253, 60.58282 ..."
2,Белоярский муниципальный округ,"POLYGON ((61.08768 56.82496, 60.9649 56.83778,..."
3,Березовский муниципальный округ,"POLYGON ((60.90269 57.21151, 60.88945 57.17303..."
4,Екатеринбург,"POLYGON ((60.01103 56.80777, 60.00708 56.80587..."


In [ ]:
# replacements = {
#     'Соколиная Гора': 'Соколиная гора',
#     'Марьина Роща': 'Марьина роща',
#     'Филевский Парк': 'Филевский парк',
#     'Нагатинский Затон': 'Нагатинский затон',

# }
# mun_geo['mun_district'] = mun_geo['mun_district'].replace(replacements)

In [42]:
# Добавляем координаты в итоговый датасет
final = pd.merge(result, mun_geo[["mun_district", "geometry"]], on="mun_district", how="left")
# final = final.drop("indicator_value", axis=1)
final

,mun_district,area,population,pop_dense,geometry
0,Академический,46.01,125000,2716.800696,"POLYGON ((60.48204 56.76587, 60.48775 56.76335..."
1,Верх-Исетский,219.79,240822,1095.691342,"POLYGON ((60.01103 56.80777, 60.00708 56.80587..."
2,Железнодорожный,125.65,158675,1262.833267,"POLYGON ((60.27079 56.84824, 60.27165 56.8483,..."
3,Кировский,86.25,220749,2559.408696,"POLYGON ((60.60165 56.84262, 60.60326 56.84341..."
4,Ленинский,22.19,222258,10016.133393,"POLYGON ((60.50303 56.75753, 60.50418 56.75659..."
5,Октябрьский,158.60,151775,956.967213,"POLYGON ((60.61146 56.83923, 60.61246 56.83599..."
6,Орджоникидзевский,99.30,263820,2656.797583,"POLYGON ((60.51112 56.93945, 60.51211 56.93921..."
7,Чкаловский,389.81,286277,734.401375,"POLYGON ((60.16854 56.74734, 60.16784 56.7457,..."


In [43]:
# Все, что осталось - Новая Москва, так что неважно
# final[final["geometry"].isna()]
final.to_csv("mun_geo_params_ekb.csv")